In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import itertools
import matplotlib.dates as mdates
from sklearn.metrics import mean_squared_error
import pdb

## cleaning & aggregation

In [1]:
def cleanstations(list_of_csvs, stations_to_drop):
    for i in np.arange(0,len(list_of_csvs)):
        try:
            list_of_csvs[i] = list_of_csvs[i].replace('(02102) Cardio Infantíl','(02102) Calle 161')
            list_of_csvs[i] = list_of_csvs[i].replace('(03009) SHAIO','(03009) Av. Suba- Calle 116')
            list_of_csvs[i] = list_of_csvs[i].replace('(05102) Mundo Aventura','(05102) Av. Américas - Av. Boyacá')
            list_of_csvs[i] = list_of_csvs[i].replace('(06106) Corferias','(06106) Recinto Ferial')
            list_of_csvs[i] = list_of_csvs[i].replace('(06108) Plaza de la Democracia','(06108) Concejo de Bogotá')
            list_of_csvs[i] = list_of_csvs[i].replace('(07105) COLISEO','(07105) MOVISTAR ARENA')
            list_of_csvs[i] = list_of_csvs[i].replace('(09115) Profamilia','(09115) Calle 34')
            list_of_csvs[i] = list_of_csvs[i].replace('(10010) Hospitales','(10010) San Bernardo')
        except:
            continue
    for j in np.arange(0,len(list_of_csvs)):
        for k in np.arange(0,len(stations_to_drop)):
            try:
                list_of_csvs[j] = list_of_csvs[j][list_of_csvs[j]['Estación'] != stations_to_drop[k]]
            except:
                continue
    
    return list_of_csvs

In [ ]:
# NOT USED
# sortbystations_dailydata and aggregation_dailydata is for the day by day CSV files

def sortbystations_dailydata(csv):
    
    '''function to sort all rides by station 
    for daily and weekly aggregation, we need to combine multiple days worth of CSVs beforehand 
    what's the column name for Stations ? it's not Acceso_Estacion'''
    
    stations = csv["Acceso_Estacion"].unique() 
    grouped = csv.groupby(["Acceso_Estacion"]).count()
    max_rides = max(grouped.iloc[:,1])
    collection_df = pd.DataFrame()
    for i in np.arange(0, len(stations)-1):
        df_station = csv.loc[csv["Acceso_Estacion"] == stations[i]]
        times = df_station["Fecha_Transaccion"].tolist()
        title = stations[i]
        if not len(times) == max_rides:
            times.extend(['']*(max_rides-len(times)))
        collection_df[title] = times
    return collection_df

def aggregation_dailydata(collection_df_onecolumn, rule):
    '''
    aggregation function
    options for rule input: '15min' [by 15 min time periods], 'D' [by day], 'W' [by week], etc
    options for collection_df_onecolumn input: select single column from collection_df (output of sortbystations function) to 
                                            see trips through one station OR input whole CSV to see trips through whole system'''
    datetime = pd.to_datetime(collection_df_onecolumn)
    frequency = pd.Series(np.ones(np.shape(collection_df_onecolumn)))
    formatted_timefreq = pd.concat([datetime, frequency], axis=1)
    formatted_timefreq.columns = ['DateTime', 'Frequency']
    formatted_timefreq = formatted_timefreq.set_index('DateTime')
    aggregated = formatted_timefreq['Frequency'].resample(rule).sum()
    return aggregated

In [2]:
# this aggregation function is for the monthly summary CSV files
# aggregates by station

# in progress

def aggregation_monthlydata(month_csv, days_in_month, stations):
    fifteen_min = {}
    daily = {}
    weekly = {}
    stations_in_data = month_csv["Estación"].unique()
    
    for station in stations:
        if station in stations_in_data:
            station_grouped = month_csv[month_csv["Estación"] == station].groupby('Intervalo').sum().iloc[:, 0:days_in_month]
            rough_agg = station_grouped.transpose().stack().reset_index()
            rough_agg = rough_agg.rename(columns={'level_0': 'date', 'Intervalo': '15 min interval', 0: 'frequency'})

            # converting the dates from strings to datetime format
            date_times = []
            for i in np.arange(0,len(rough_agg)):
                day = rough_agg['date'][i].strftime("%Y-%m-%d")
                date_string = day + rough_agg['15 min interval'][i]
                date_times.append(datetime.strptime(date_string, '%Y-%m-%d%H:%M'))

            rough_agg['datetime'] = date_times
            aggregated_15min = rough_agg.set_index('datetime').drop(['date','15 min interval'],axis=1)
            aggregated_day = aggregated_15min.resample('D').sum()
            aggregated_week = aggregated_15min.resample('W').sum()

            fifteen_min[station] = aggregated_15min
            daily[station] = aggregated_day
            weekly[station] = aggregated_week
            
    return [fifteen_min, daily, weekly]

## creating datasets

In [ ]:
def train_test_covid(stations, agg_type):
    data_dict = {}
    for i in np.arange(0,len(stations)):
        s = stations[i]
        training = pd.concat([jan2018_agg[agg_type][s], feb2018_agg[agg_type][s], march2018_agg[agg_type][s], 
                              april2018_agg[agg_type][s], may2018_agg[agg_type][s], june2018_agg[agg_type][s],
                              july2018_agg[agg_type][s], aug2018_agg[agg_type][s], sept2018_agg[agg_type][s], 
                              oct2018_agg[agg_type][s], nov2018_agg[agg_type][s], dec2018_agg[agg_type][s],
                              jan2019_agg[agg_type][s], feb2019_agg[agg_type][s], march2019_agg[agg_type][s], 
                              april2019_agg[agg_type][s], may2019_agg[agg_type][s], june2019_agg[agg_type][s],
                              july2019_agg[agg_type][s], aug2019_agg[agg_type][s], sept2019_agg[agg_type][s], 
                              oct2019_agg[agg_type][s], nov2019_agg[agg_type][s], dec2019_agg[agg_type][s]])
        test = pd.concat([jan2020_agg[agg_type][s], feb2020_agg[agg_type][s]])
        covid = pd.concat([march2020_agg[agg_type][s], april2020_agg[agg_type][s], may2020_agg[agg_type][s]])
        data_dict[s] = {'training': training,
                        'test': test,
                        'COVID': covid}
        
    return data_dict

In [ ]:
# function that displays regular plot, autocorrelation plot, and partial autocorrelation plot for each level of differencing
# use autocorrelation plot to decide differencing (d) and AR (p)
# use partial autocorrelation plot to decide MA (q)
# note: AR factors correct for slight under-differencing, MA factors correct for slight over-differencing

def differencing_acf_pacf_plots(aggregated):
    fig, axes = plt.subplots(3, 3, figsize=(20,15))
    axes[0, 0].plot(aggregated); axes[0, 0].set_title('Original Series')
    plot_acf(aggregated, ax=axes[0, 1])
    plot_pacf(aggregated, ax=axes[0, 2])

    # 1st Differencing
    axes[1, 0].plot(aggregated.diff()); axes[1, 0].set_title('1st Order Differencing')
    plot_acf(aggregated.diff().dropna(), ax=axes[1, 1])
    plot_pacf(aggregated.diff().dropna(), ax=axes[1, 2])

    # 2nd Differencing
    axes[2, 0].plot(aggregated.diff().diff()); axes[2, 0].set_title('2nd Order Differencing')
    plot_acf(aggregated.diff().diff().dropna(), ax=axes[2, 1])
    plot_pacf(aggregated.diff().diff().dropna(), ax=axes[2, 2])

    plt.show()
    return

In [ ]:
def prediction_residual_plots(test_and_predictions_TESTdata, test_and_predictions_COVIDdata, figure_title=''):
    fig, ax = plt.subplots(2, 2, sharey='row', figsize=(20, 15))
    fig.suptitle(figure_title, fontsize=16)
    
    #Jan-Feb 2020
    ax[0,0].plot(test_and_predictions_TESTdata['observed'], label='observed')
    ax[0,0].plot(test_and_predictions_TESTdata['predictions'], label='predicted')
    ax[0,0].set(xlabel="Date",
           ylabel="Ridership",
           title='July 2019-Feb 2020')
    date_form = mdates.DateFormatter("%b-%d-%y")
    ax[0,0].xaxis.set_major_formatter(date_form)
    ax[0,0].legend()
    
    errors = test_and_predictions_TESTdata['predictions'] - test_and_predictions_TESTdata['observed']
    ax[1,0].scatter(test_and_predictions_TESTdata.index, pd.DataFrame(errors).set_index(test_and_predictions_TESTdata.index))
    ax[1,0].plot(test_and_predictions_TESTdata.index, np.zeros(len(test_and_predictions_TESTdata.index)))
    ax[1,0].set(xlabel="Date",
           ylabel="Residuals",
           title= 'Residual Distribution')
    ax[1,0].xaxis.set_major_formatter(date_form)
    
    #March-July 2020
    ax[0,1].plot(test_and_predictions_COVIDdata['observed'], label='observed')
    ax[0,1].plot(test_and_predictions_COVIDdata['predictions'], label='predicted')
    ax[0,1].set(xlabel="Date",
           ylabel="Ridership",
           title='March 2020-April 2021')
    ax[0,1].xaxis.set_major_formatter(date_form)
    ax[0,1].legend()
    
    errors = test_and_predictions_COVIDdata['predictions'] - test_and_predictions_COVIDdata['observed']
    ax[1,1].scatter(test_and_predictions_COVIDdata.index, pd.DataFrame(errors).set_index(test_and_predictions_COVIDdata.index))
    ax[1,1].plot(test_and_predictions_COVIDdata.index, np.zeros(len(test_and_predictions_COVIDdata.index)))
    ax[1,1].set(xlabel="Date",
           ylabel="Residuals",
           title= 'Residual Distribution')
    ax[1,1].xaxis.set_major_formatter(date_form)
            
    return

## rolling ARIMA/SARIMA functions

In [ ]:
# function to apply ARIMA to aggregated dataset, returns plot of predicted vs actual test dataset

def applyARIMA_rolling(agg_training, agg_test, order=(2,0,0), plot=True, plot_title='', resid_plot=True):
    train = agg_training.values
    test = agg_test.values
    history = [x for x in train]
    predictions = list()
    if agg_test.index[0] == datetime(2020,3,1):
        for t in range(len(test) + 31 + 29):
            model = ARIMA(history, order=order)
            model_fit = model.fit()
            output = model_fit.forecast()
            yhat = output[0]
            predictions.append(yhat)
            if t < 60:
                history.append(np.array([yhat]))
            else:
                obs = test[t-60]
                history.append(obs)
        test_and_predictions = pd.DataFrame({'observed': test.flatten(), 'predictions': predictions[60:]}).set_index(agg_test.index)
    else:
        for t in range(len(test)):
            model = ARIMA(history, order=order)
            model_fit = model.fit()
            output = model_fit.forecast()
            yhat = output[0]
            predictions.append(yhat)
            obs = test[t]
            history.append(obs)
        test_and_predictions = pd.DataFrame({'observed': test.flatten(), 'predictions': predictions}).set_index(agg_test.index)
    
    if plot==True:
        fig, ax = plt.subplots(figsize=(10, 8))
        ax.plot(test_and_predictions['observed'], label='observed')
        ax.plot(test_and_predictions['predictions'], label='prediction')
        ax.set(xlabel="Date",
               ylabel="Ridership",
               title=plot_title)
        date_form = mdates.DateFormatter("%b-%d")
        ax.xaxis.set_major_formatter(date_form)
        plt.legend()
        plt.show()
    
    if resid_plot==True:
        errors = test_and_predictions['predictions'] - test_and_predictions['observed']
        fig, ax = plt.subplots(figsize=(10,8))
        ax.scatter(agg_test.index, pd.DataFrame(errors).set_index(agg_test.index))
        ax.plot(agg_test.index, np.zeros(len(agg_test.index)))
        date_form = mdates.DateFormatter("%b-%d")
        ax.xaxis.set_major_formatter(date_form)
        plt.show()
        
    return {'observed and predicted values': test_and_predictions, 'model object': model_fit}

# function to apply SARIMA to aggregated dataset, returns plot of predicted vs actual test dataset

def applySARIMA_rolling(agg_training, agg_test, order=(2,0,0), seasonal_order=(0,0,0,0), plot=True, plot_title='', resid_plot=True):
    train = agg_training.values
    test = agg_test.values
    history = [x for x in train]
    predictions = list()
    if agg_test.index[0] == datetime(2020,3,1):
        for t in range(len(test) + 31 + 29):
            #pdb.set_trace()
            model = SARIMAX(history, order=order, seasonal_order=seasonal_order, initialization='approximate_diffuse')
            model_fit = model.fit()
            output = model_fit.forecast()
            yhat = output[0]
            predictions.append(yhat)
            if t < 60:
                history.append(np.array([yhat]))
            else:
                obs = test[t-60]
                history.append(obs)
        test_and_predictions = pd.DataFrame({'observed': test.flatten(), 'predictions': predictions[60:]}).set_index(agg_test.index)
    else:
        for t in range(len(test)):
            model = SARIMAX(history, order=order, seasonal_order=seasonal_order)
            model_fit = model.fit() 
            output = model_fit.forecast()
            yhat = output[0]
            predictions.append(yhat)
            obs = test[t]
            history.append(obs)
        test_and_predictions = pd.DataFrame({'observed': test.flatten(), 'predictions': predictions}).set_index(agg_test.index)
    
    if plot==True:
        fig, ax = plt.subplots(figsize=(10, 8))
        ax.plot(test_and_predictions['observed'], label='observed')
        ax.plot(test_and_predictions['predictions'], label='prediction')
        ax.set(xlabel="Date",
               ylabel="Ridership",
               title=plot_title)
        date_form = mdates.DateFormatter("%b-%d")
        ax.xaxis.set_major_formatter(date_form)
        plt.legend()
        plt.show()
    if resid_plot==True:
        errors = test_and_predictions['predictions'] - test_and_predictions['observed']
        fig, ax = plt.subplots(figsize=(10,8))
        ax.scatter(agg_test.index, pd.DataFrame(errors).set_index(agg_test.index))
        ax.plot(agg_test.index, np.zeros(len(agg_test.index)))
        date_form = mdates.DateFormatter("%b-%d")
        ax.xaxis.set_major_formatter(date_form)
        plt.show()
    
    return {'observed and predicted values': test_and_predictions, 'model object': model_fit}

## static ARIMA/SARIMA functions

In [5]:
def applyARIMA_static(agg_training, agg_test, order=(2,0,0), plot=True, plot_title='', resid_plot=True):
    train = agg_training.values
    test = agg_test.values
    model = ARIMA(train, order=order)
    model_fit = model.fit()
    
    if agg_test.index[0] == datetime(2020,3,1):
        predictions = model_fit.forecast(steps=len(test) + 31 + 29)
        predictions = predictions[60:]
    else:
        predictions = model_fit.forecast(steps=len(test))
    test_and_predictions = pd.DataFrame({'observed': test.flatten(), 'predictions': predictions}).set_index(agg_test.index)
    
    if plot==True:
        fig, ax = plt.subplots(figsize=(10, 8))
        ax.plot(test_and_predictions['observed'], label='observed')
        ax.plot(test_and_predictions['predictions'], label='prediction')
        ax.set(xlabel="Date",
               ylabel="Ridership",
               title=plot_title)
        date_form = mdates.DateFormatter("%b-%d")
        ax.xaxis.set_major_formatter(date_form)
        plt.legend()
        plt.show()
    if resid_plot==True:
        errors = test_and_predictions['predictions'] - test_and_predictions['observed']
        fig, ax = plt.subplots(figsize=(10,8))
        ax.scatter(agg_test.index, pd.DataFrame(errors).set_index(agg_test.index))
        ax.plot(agg_test.index, np.zeros(len(agg_test.index)))
        date_form = mdates.DateFormatter("%b-%d")
        ax.xaxis.set_major_formatter(date_form)
        plt.show()
        
    return {'observed and predicted values': test_and_predictions, 'model object': model_fit}

def applySARIMA_static(agg_training, agg_test, order=(2,0,0), seasonal_order=(0,0,0,0), plot=True, plot_title='', resid_plot=True):
    train = agg_training.values
    test = agg_test.values
    model = SARIMAX(train, order=order, seasonal_order=seasonal_order)
    model_fit = model.fit()
    
    if agg_test.index[0] == datetime(2020,3,1):
        predictions = model_fit.forecast(steps=len(test) + 31 + 29)
        predictions = predictions[60:]
    else:
        predictions = model_fit.forecast(steps=len(test))
    test_and_predictions = pd.DataFrame({'observed': test.flatten(), 'predictions': predictions}).set_index(agg_test.index)
    
    if plot==True:
        fig, ax = plt.subplots(figsize=(10, 8))
        ax.plot(test_and_predictions['observed'], label='observed')
        ax.plot(test_and_predictions['predictions'], label='prediction')
        ax.set(xlabel="Date",
               ylabel="Ridership",
               title=plot_title)
        date_form = mdates.DateFormatter("%b-%d")
        ax.xaxis.set_major_formatter(date_form)
        plt.legend()
        plt.show()
    if resid_plot==True:
        errors = test_and_predictions['predictions'] - test_and_predictions['observed']
        fig, ax = plt.subplots(figsize=(10,8))
        ax.scatter(agg_test.index, pd.DataFrame(errors).set_index(agg_test.index))
        ax.plot(agg_test.index, np.zeros(len(agg_test.index)))
        date_form = mdates.DateFormatter("%b-%d")
        ax.xaxis.set_major_formatter(date_form)
        plt.show()
    return {'observed and predicted values': test_and_predictions, 'model object': model_fit}

## intermediate ARIMA/SARIMA functions

In [ ]:
def applySARIMA_intermediate(agg_training, agg_test, order=(2,0,0), seasonal_order=(0,0,0,0), agg_type='daily', plot=True, plot_title='', resid_plot=True):
    train = agg_training
    test = agg_test
    predictions = []
    
    if agg_type=='15 min':
        pred_window = 4
    elif agg_type=='daily':
        pred_window = 7
    elif agg_type=='weekly':
        pred_window = 4
    
    model = SARIMAX(train, order=order, seasonal_order=seasonal_order)
    model_fit = model.fit()
    for x in model_fit.forecast(pred_window).tolist():
        predictions.append(x)
    
    k=0
    while pred_window*k+(pred_window-1) < len(test.values):
        week_startdate = test.index[pred_window*k].strftime('%Y-%m-%d')
        week_enddate = test.index[pred_window*k+(pred_window-1)].strftime('%Y-%m-%d')
        model_fit = model_fit.append(test[week_startdate:week_enddate])
        for x in model_fit.forecast(pred_window).tolist():
            predictions.append(x)
        k=k+1
    
    test_and_predictions = pd.DataFrame({'observed': test.values.flatten(), 'predictions': predictions[:len(test.values)]}).set_index(test.index)
    
    if plot==True:
        fig, ax = plt.subplots(figsize=(10, 8))
        ax.plot(test_and_predictions['observed'], label='observed')
        ax.plot(test_and_predictions['predictions'], label='prediction')
        ax.set(xlabel="Date",
               ylabel="Ridership",
               title=plot_title)
        date_form = mdates.DateFormatter("%b-%d")
        ax.xaxis.set_major_formatter(date_form)
        plt.legend()
        plt.show()
    if resid_plot==True:
        errors = test_and_predictions['predictions'] - test_and_predictions['observed']
        fig, ax = plt.subplots(figsize=(10,8))
        ax.scatter(agg_test.index, pd.DataFrame(errors).set_index(agg_test.index))
        ax.plot(agg_test.index, np.zeros(len(agg_test.index)))
        date_form = mdates.DateFormatter("%b-%d")
        ax.xaxis.set_major_formatter(date_form)
        plt.show()
        
    return {'observed and predicted values': test_and_predictions, 'model object': model_fit}

## evaluating models / gridsearch

In [ ]:
def evaluate_models_ARIMA(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_order = float("0"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    test_and_predictions, model_fit = applyARIMA(dataset,order=order,plot=False)
                    corr = test_and_predictions.corr().iloc[1,0]
                    if corr > best_score:
                        best_score, best_order = corr, order
                    #print('ARIMA%s R^2=%.3f' % (order,corr))
                except:
                    continue
    print('Best ARIMA%s R^2=%.3f' % (best_order, best_score))

In [ ]:
def create_random_sample_from_parameters(hyperparam_dict, size = 10):
    ''' Returns a DataFrame with len size of random draws from the grid. 
    Input:
    hyperparam_dict: dictionary with keys p/d/q and P/D/Q/m, if applicable 
    size = Number of random draws
    '''
    if len(hyperparam_dict)==3:
        p = hyperparam_dict['p']
        d = hyperparam_dict['d']
        q = hyperparam_dict['q']
        all_param_combinations = itertools.product(p,d,q)
        column_labels = ['p','d','q']
    if len(hyperparam_dict)==7:
        p = hyperparam_dict['p']; d = hyperparam_dict['d']; q = hyperparam_dict['q']
        P = hyperparam_dict['P']
        D = hyperparam_dict['D']
        Q = hyperparam_dict['Q']
        m = hyperparam_dict['m']
        all_param_combinations = list(itertools.product(p,d,q,P,D,Q,m))
        column_labels = ['p','d','q','P','D','Q','m']
        
    grid = pd.DataFrame(all_param_combinations, columns=column_labels)
    grid_rands = grid.iloc[np.random.choice(grid.index, size, replace=False),:] 
    return grid_rands

In [4]:
def grid_search(hyperparam_dict, agg_training, agg_test, model='ARIMA', model_type='intermediate', agg_type='daily', iterations = 50):
    
    random_grid = create_random_sample_from_parameters(hyperparam_dict, size = iterations)
    score_list = []
    normBIC_list = []
    MAPE_list = []
    
    for x in np.arange(0,len(random_grid)):
        #print(x)
        if model=='ARIMA':
            order = random_grid.iloc[x,:]
            try:
                if model_type=='rolling':
                    output = applyARIMA_rolling(agg_training,agg_test,order=order,plot=False,resid_plot=False)
                elif model_type=='static':
                    output = applyARIMA_static(agg_training,agg_test,order=order,plot=False,resid_plot=False)
                
                test_and_predictions = output['observed and predicted values']
                
                corr = test_and_predictions.corr().iloc[1,0] ** 2
                score_list.append(corr)
                MSE = mean_squared_error(test_and_predictions['observed'],test_and_predictions['predictions'])
                normBIC = np.log(MSE) + sum(order+1)*np.log(output['model object'].nobs)/output['model object'].nobs
                normBIC_list.append(normBIC)
                MAPE_list.append(MAPE(test_and_predictions))
                if normBIC == np.nanmin(normBIC_list):
                    final_hp = order
                    final_output = output
            except:
                score_list.append(np.nan)
                normBIC_list.append(np.nan)
                MAPE_list.append(np.nan)
                continue
        elif model=='SARIMA':
            order = random_grid.iloc[x,0:3]
            s_order = random_grid.iloc[x,3:8]
            try:
                if model_type=='rolling':
                    output = applySARIMA_rolling(agg_training,agg_test,order=order,seasonal_order=s_order,plot=False,resid_plot=False)
                elif model_type=='static':
                    output = applySARIMA_static(agg_training,agg_test,order=order,seasonal_order=s_order,plot=False,resid_plot=False)
                elif model_type=='intermediate':
                    output = applySARIMA_intermediate(agg_training, agg_test, order=order, seasonal_order=s_order, agg_type=agg_type, plot=False, resid_plot=False)
                
                test_and_predictions = output['observed and predicted values']
                
                #pdb.set_trace()
                try:
                    MSE = mean_squared_error(test_and_predictions['observed'],test_and_predictions['predictions'])
                    normBIC = np.log(MSE) + sum(order+1)*np.log(output['model object'].nobs)/output['model object'].nobs
                    normBIC_list.append(normBIC)
                    MAPE_list.append(MAPE(test_and_predictions))
                    corr = test_and_predictions.corr().iloc[1,0] ** 2
                    score_list.append(corr)
                    if normBIC == np.nanmin(normBIC_list):
                        final_hp = [order, s_order]
                        final_output = output
                except ValueError: 
                    normBIC_list.append(np.nan)
                    MAPE_list.append(np.nan)
                    score_list.append(np.nan)
                    continue
            except:
                normBIC_list.append(np.nan)
                MAPE_list.append(np.nan)
                score_list.append(np.nan)
                continue
    
    results = random_grid
    results['R-squared'] = score_list
    results['normalized BIC'] = normBIC_list
    results['MAPE'] = MAPE_list
    
    return {'final hyperparams': final_hp, 
            'observed and predicted values': final_output['observed and predicted values'],
            'model object': final_output['model object'],
            'gridsearch results': results}
    #return results

In [14]:
# MAPE function
# filters out time periods with 0 actual trips and replaces negative predicted trips with 0

def MAPE(test_and_predictions):
    predictions = test_and_predictions['predictions']
    for i in np.arange(0,len(predictions)-1):
        if predictions[i] < 0:
            predictions[i] = 0
    data = pd.DataFrame({'observed': test_and_predictions['observed'],
                        'predictions': predictions})
    data = data[data['observed'] != 0]
    return np.mean(np.abs((data['observed'] - data['predictions']) / data['observed'])) * 100

In [13]:
def VAPE(test_and_predictions):
    predictions = test_and_predictions['predictions']
    for i in np.arange(0,len(predictions)-1):
        if predictions[i] < 0:
            predictions[i] = 0
    errors = test_and_predictions['observed'] - predictions
    return np.mean((errors - np.mean(errors))**2) * 100

In [ ]:
def RMSE(test_and_predictions):
    predictions = test_and_predictions['predictions']
    for i in np.arange(0,len(predictions)-1):
        if predictions[i] < 0:
            predictions[i] = 0
    data = test_and_predictions
    data = data[data['observed'] != 0]
    return np.sqrt(np.mean((data['observed'] - data['predictions'])**2))

## automating process - one station

In [5]:
def rollingSARIMA_results(cleaned_data_onestation, hyperparam_dict, agg_type='daily', log_transform=True):
    fifteen_min = cleaned_data_onestation
    daily = cleaned_data_onestation.resample('D').sum()
    weekly = cleaned_data_onestation.resample('W').sum()
    
    if agg_type=='15 min':
        train = fifteen_min.loc['2018-01-01':'2019-12-31'] # August 2015 - June 2019
        test = fifteen_min.loc['2020-01-01':'2020-02-29'] # July 2019 - Feb 2020
        covid = fifteen_min.loc['2020-03-01':'2020-05-31'] # March 2020 - April 2021
    elif agg_type=='daily':
        train = daily.loc['2018-01-01':'2019-12-31']
        test = daily.loc['2020-01-01':'2020-02-29']
        covid = daily.loc['2020-03-01':'2020-05-31']
    elif agg_type=='weekly':
        train = weekly.loc['2018-01-01':'2019-12-31']
        test = weekly.loc['2020-01-01':'2020-02-29']
        covid = weekly.loc['2020-03-01':'2020-05-31']
    else:
        print('options for agg_type: 15 min, daily, weekly')
        return
    
    if log_transform==True:
        train = np.log(train)
        test = np.log(test)
        covid = np.log(covid)
    
    results_test = grid_search(hyperparam_dict, train, test, model='SARIMA', model_type='rolling', iterations = 10)
    best_order = results_test['final hyperparams'][0].values
    best_s_order = results_test['final hyperparams'][1].values
    
    results_covid = applySARIMA_rolling(train, covid, order=best_order, seasonal_order=best_s_order, plot=False, resid_plot=False)
    
    fig_title = cleaned_data_onestation.columns[0] + '\nSARIMA' + str(best_order) + str(best_s_order)
    prediction_residual_plots(results_test['observed and predicted values'], results_covid['observed and predicted values'], figure_title=fig_title)
    
    return {'final hyperparams': results_test['final hyperparams'],
           'predicted values (test)': results_test['observed and predicted values'],
           'model object (test)': results_test['model object'],
           'gridsearch results': results_test['gridsearch results'],
           'predicted values (COVID)': results_covid['observed and predicted values'],
           'model object (COVID)': results_covid['model object']}
    
    #return [results_test, results_covid]

In [ ]:
def staticSARIMA_results(cleaned_data_onestation, hyperparam_dict, agg_type='daily', log_transform=True):
    fifteen_min = cleaned_data_onestation
    daily = cleaned_data_onestation.resample('D').sum()
    weekly = cleaned_data_onestation.resample('W').sum()
    
    if agg_type=='15 min':
        train = fifteen_min.loc['2015-08-01':'2019-06-30'] # August 2015 - June 2019
        test = fifteen_min.loc['2019-07-01':'2020-02-29'] # July 2019 - Feb 2020
        covid = fifteen_min.loc['2020-03-01':'2021-04-30'] # March 2020 - April 2021
    elif agg_type=='daily':
        train = daily.loc['2015-08-01':'2019-06-30']
        test = daily.loc['2019-07-01':'2020-02-29']
        covid = daily.loc['2020-03-01':'2021-04-30']
    elif agg_type=='weekly':
        train = weekly.loc['2015-08-01':'2019-06-30']
        test = weekly.loc['2019-07-01':'2020-02-29']
        covid = weekly.loc['2020-03-01':'2021-04-30']
    else:
        print('options for agg_type: 15 min, daily, weekly')
        return
    
    if log_transform==True:
        train = np.log(train+1)
        test = np.log(test+1)
        covid = np.log(covid+1)
    
    results_test = grid_search(hyperparam_dict, train, test, model='SARIMA', model_type='static', iterations = 10)
    best_order = results_test['final hyperparams'][0].values
    best_s_order = results_test['final hyperparams'][1].values
    
    results_covid = applySARIMA_static(train, covid, order=best_order, seasonal_order=best_s_order, plot=False, resid_plot=False)
    
    fig_title = cleaned_data_onestation.columns[0] + '\nSARIMA' + str(best_order) + str(best_s_order)
    prediction_residual_plots(np.exp(results_test['observed and predicted values']), np.exp(results_covid['observed and predicted values']), figure_title=fig_title)
    
    return {'final hyperparams': results_test['final hyperparams'],
           'predicted values (test)': results_test['observed and predicted values'],
           'model object (test)': results_test['model object'],
           'gridsearch results': results_test['gridsearch results'],
           'predicted values (COVID)': results_covid['observed and predicted values'],
           'model object (COVID)': results_covid['model object']}

In [2]:
def intermediateSARIMA_results(cleaned_data_onestation, hyperparam_dict, agg_type='daily', log_transform=True):
    fifteen_min = cleaned_data_onestation
    daily = cleaned_data_onestation.resample('D').sum()
    weekly = cleaned_data_onestation.resample('W').sum()
    
    if agg_type=='15 min':
        train = fifteen_min.loc['2015-08-01':'2019-06-30'] # August 2015 - June 2019
        test = fifteen_min.loc['2019-07-01':'2020-02-29'] # July 2019 - Feb 2020
        covid = fifteen_min.loc['2020-03-01':'2021-04-30'] # March 2020 - April 2021
    elif agg_type=='daily':
        train = daily.loc['2015-08-01':'2019-06-30']
        test = daily.loc['2019-07-01':'2020-02-29']
        covid = daily.loc['2020-03-01':'2021-04-30']
    elif agg_type=='weekly':
        train = weekly.loc['2015-08-01':'2019-06-30']
        test = weekly.loc['2019-07-01':'2020-02-29']
        covid = weekly.loc['2020-03-01':'2021-04-30']
    else:
        print('options for agg_type: 15 min, daily, weekly')
        return
    
    if log_transform==True:
        train = np.log(train+1)
        test = np.log(test+1)
        covid = np.log(covid+1)
    
    results_test = grid_search(hyperparam_dict, train, test, model='SARIMA', model_type='intermediate', agg_type='weekly',iterations = 10)
    best_order = results_test['final hyperparams'][0].values
    best_s_order = results_test['final hyperparams'][1].values
    
    results_covid = applySARIMA_intermediate(train, pd.concat([test,covid]), order=best_order, seasonal_order=best_s_order, agg_type=agg_type, plot=False, resid_plot=False)
    
    fig_title = cleaned_data_onestation.columns[0] + '\nSARIMA' + str(best_order) + str(best_s_order)
    prediction_residual_plots(np.exp(results_test['observed and predicted values']), np.exp(results_covid['observed and predicted values']['2020-03-01':'2021-04-30']), figure_title=fig_title)
    
    return {'final hyperparams': results_test['final hyperparams'],
           'predicted values (test)': results_test['observed and predicted values'],
           'model object (test)': results_test['model object'],
           'gridsearch results': results_test['gridsearch results'],
           'predicted values (COVID)': results_covid['observed and predicted values'],
           'model object (COVID)': results_covid['model object']}